# 5장 반복 찾기

## 몇 번 일치하는가?  
### 하나 이상의 문자 찾기
* + 문자로 '하나 이상'을 표현할 수 있다. 즉, a+는 하나 이상 연속된 a를 찾는다. [0-9]+ 는 한 자리 이상 연속된 숫자를 찾는다. ([0-9+]는 숫자 0부터 9 그리고 더하기 기호(+)를 찾을 뿐이다.)  
* 더하기(+)는 메타 문자이므로 더하기 문자 자체를 찾으려면 \+로 해야 한다.

ex)  
\w+@\w+\.\w+  
하지만 이 경우에는  
ben.forta@urgent.forta.com과 같은 이메일은 찾을 수 없다. \w는 영숫자와 일치하나, 마침표와는 일치하지 않기에, @ 앞의 마침표를 검색할 수 없다. 또한 정규표현식으로 @ 뒤의 마침표를 하나만 검색하도록 설정했기 때문에, @ 뒤의 경우도 적절치 못하다.  

이런 경우 집합을 만들고 +를 해주면 된다.  
\w+[\w.]+@[\w.]+\.\w+  
  
* 메타 문자가 집합의 구성원일 경우엔, 문자 그대로 취급되므로 굳이 이스케이프할 필요 없다. 그러나 해준다고 해서 문제가 생기지는 않는다. 즉, [\w.]과 [\w\.]은 같다.

### 문자가 없거나 하나 이상 연속하는 문자 찾기
* 더하기(+)는 하나 이상 연속된 문자를 찾는다. 즉, 문자가 없는 경우는 아예 찾지 못하고, 최소한 하나는 일치해야 한다.
* 있을 수도 있고 없을 수도 있는 문자를 찾으려면 별표(*)를 사용한다.  
위 정규 표현식을 \w+[\w.]*@[\w.]+\.\w+ 로 바꿔줄 수 있겠다.  

In [34]:
import re

ex = '''a@b.c
ben@forta.com  
support@forta.com  
spam@forta.com
ben.forta@urgent.forta.com
.b@forta.com'''
p1 = re.compile(r'\w@\w\.\w')
p2 = re.compile(r'\w+@\w+\.\w+')
p3 = re.compile(r'\w+[\w.]+@[\w.]+\.\w+')
p4 = re.compile(r'\w+[\w.]*@[\w.]+\.\w+')

print(p1.findall(ex))
print(p2.findall(ex))
print(p3.findall(ex))
print(p4.findall(ex))

['a@b.c']
['a@b.c', 'ben@forta.com', 'support@forta.com', 'spam@forta.com', 'forta@urgent.forta', 'b@forta.com']
['ben@forta.com', 'support@forta.com', 'spam@forta.com', 'ben.forta@urgent.forta.com']
['a@b.c', 'ben@forta.com', 'support@forta.com', 'spam@forta.com', 'ben.forta@urgent.forta.com', 'b@forta.com']


### 문자가 없거나 하나인 문자
* 메타 문자인 물음표(?)는 자기 앞에 있는 문자가 없거나 하나만 있는 경우에 일치한다.
* 정규 표현식 https*:\/\/[\w.\/]+ 는 httpsssss://와도 일치하므로 부적절하다. 이런 경우 https?:\/\/[\w.\/]+ 가 옳겠다. https?는 http나 https와는 일치하지만 그 외에는 일치하지 않는다.
  
* [\r]?\n은 /r이 있을 경우 \r과 일치하고, \n과는 반드시 일치한다.
* \r?이 아닌 [\r]?로 사용하는 이유는 (실제로 둘은 기능이 같다) 집합([])은 혼란을 방지하고자 문자가 하나일 때도 사용하기 때문이다. 즉 뒤에 나오는 메타 문자(?)가 어디에 사용되는지 확실하게 하기 위함이다.

In [24]:
import re

ex = '''The URL is http://www.forta.com/, to connect
securely use https://www.forta.com/ instead.
This URL httpsssssss://www.forta.com/ is also an example'''

p1 = re.compile(r'https*:\/\/[\w.\/]+')
p2 = re.compile(r'https?:\/\/[\w.\/]+')

print(p1.findall(ex))
print(p2.findall(ex))

['http://www.forta.com/', 'https://www.forta.com/', 'httpsssssss://www.forta.com/']
['http://www.forta.com/', 'https://www.forta.com/']


## 구간 지정하기
* +와 *는 일치하는 문자 수에 제한이 없다.
* 원하는 만큼의 수만 일치하도록 하려면 구간(interval)을 활용하고 그것은 {숫자}로 표현한다.

### 범위 구간
* 범위 구간은 숫자를 두 번 써서 나타낸다. \d{2,4}는 2자리~4자리를 의미한다.
* 구간은 0부터 시작할 수 있다. {0,3}은 요소가 없는 경우와 한 번, 두 번, 세 번 일치함을 의미한다.
* ?는 {0,1}과 기능이 같다고 하겠다.

### '최소' 구간
* 최댓값 없이 최솟값만 지정할 수 있다. {3,}은 최소 세 번 이상을 말한다.
* 더하기(+)는 {1,}과 같다고 하겠다.

In [36]:
import re

ex = '''body {
background-color: #fefbd8;
}
h1 {
background-color: #0000ff;
}
div {
background-color: #d0f4e6;
}
span {
background-color: #f08970;
}'''

p1 = re.compile(r'#[A-Fa-f0-9]')
p2 = re.compile(r'#[A-Fa-f0-9]{6}')

print(p1.findall(ex))
print(p2.findall(ex))

['#f', '#0', '#d', '#f']
['#fefbd8', '#0000ff', '#d0f4e6', '#f08970']


In [38]:
import re

ex = '''1001: $496.80
1002: $1290.69
1003: $26.43
1004: $613.42
1004: $613.42
1005: $7.61
1006: $414.90
1007: $25.00'''

p1 = re.compile(r'\d+: \$\d{2,}\.\d')
p2 = re.compile(r'\d+: \$\d{3,}\.\d')

print(p1.findall(ex))
print(p2.findall(ex))

['1001: $496.8', '1002: $1290.6', '1003: $26.4', '1004: $613.4', '1004: $613.4', '1006: $414.9', '1007: $25.0']
['1001: $496.8', '1002: $1290.6', '1004: $613.4', '1004: $613.4', '1006: $414.9']


## 과하게 일치하는 상황 방지
* *와 + 같은 메타 문자는 탐욕적이기 때문에 가장 큰 덩어리를 찾으려고 한다. 이런 메타 문자는 찾으려는 텍스트를 텍스트의 마지막에서 시작해 거꾸로 찾는다. 이는 의도적으로 수량자(quantifier)를 탐욕적으로 설계한 탓이다.  
  
예를 들어 정규 표현식 <[Bb]>.*<\/[Bb]>을 활용하면,  
<b>AK</b> and <b>HI</b>  
를 탐색한다.  
* 이런 경우 탐욕적 수량자를 게으른(lazy) 수량자로 변경해주면 된다. 수량자 뒤에 ?를 붙이면 된다.  
* 탐욕적 수량자 * + {n,} 게으른 수량자 *? +? {n,}?
* <[Bb]>.*<\/[Bb]>f를 <[Bb]>.*?<\/[Bb]>로 변경하면,  
<b>AK</b> <b>HI</b>만 일치한다.

In [21]:
import re

ex = '''This offer is not available to customers
living in <b>AK</b> and <b>HI</b>.'''

p1 = re.compile(r'<[Bb]>.*<\/[Bb]>')
p2 = re.compile(r'<[Bb]>.*?<\/[Bb]>')

print(p1.findall(ex))
print(p2.findall(ex))


['<b>AK</b> and <b>HI</b>']
['<b>AK</b>', '<b>HI</b>']
